## import

In [276]:
import numpy as np
import sys,os
import random
import matplotlib.pyplot as plt
import csv
import ast
from collections import OrderedDict

## Input Data 생성

x, y, z 데이터 중 

x가 0~99이면    정답 => [1,0,0]

x가 100~199이면 정답 => [0,1,0]

x가 200~299이면 정답 => [0,0,1]

In [277]:
input_data=np.empty((0,3))
t=np.empty((0,3))
for i in range(1000):
    input_data=np.append(input_data, np.array([[random.randrange(0,300) for j in range(3)]]),axis=0)
    x=input_data[-1,0]
    if x<100:
        t=np.append(t,np.array([[1, 0, 0]]),axis=0)
    elif x<200:
        t=np.append(t,np.array([[0, 1, 0]]),axis=0)
    elif x<300:
        t=np.append(t,np.array([[0, 0, 1]]),axis=0)
print(input_data.shape)

(1000, 3)


## Input Data Normalize  (0~1)

정규화 식


<img src = "../simple_network/Data/normalize.png" width = "200" height = "75">



In [278]:
input_max=299
input_min=0
de=input_max-input_min
normalize_input=np.zeros((1000,3),dtype=float)
for i in range(1000):
    for j in range(3):
        normalize_input[i,j]=(input_data[i,j]-input_min)/de
print(input_data)
print(normalize_input)

[[ 38.  64. 270.]
 [155. 111. 173.]
 [260. 133. 248.]
 ...
 [ 27.  49.  98.]
 [111. 174. 107.]
 [208. 290. 197.]]
[[0.1270903  0.21404682 0.90301003]
 [0.51839465 0.37123746 0.57859532]
 [0.86956522 0.44481605 0.82943144]
 ...
 [0.090301   0.1638796  0.3277592 ]
 [0.37123746 0.5819398  0.35785953]
 [0.69565217 0.96989967 0.65886288]]


### Input Data 파일에 저장

In [290]:
np.savez("../simple_network/Data/input_data_set.npz", input_data)
np.savez("../simple_network/Data/t_set.npz", t)


with open('input.csv', 'w', newline='') as csvfile:
    # Create a CSV writer
    writer = csv.writer(csvfile)

    # Write the rows of the list to the CSV file
    for row in input_data:
        writer.writerow(row)

with open('ttt.csv', 'w', newline='') as csvfile:
    # Create a CSV writer
    writer = csv.writer(csvfile)

    # Write the rows of the list to the CSV file
    for row in t:
        writer.writerow(row)

#### Input Data 파일 읽기 테스트

np.savez()를 사용하여 numpy array 형 변수 읽어오기


In [280]:
read_input=np.load("../simple_network/Data/input_data_set.npz", allow_pickle=True)
read_t=np.load("../simple_network/Data/t_set.npz", allow_pickle=True)
r_input_data=read_input["arr_0"]
r_t_data=read_t["arr_0"]
print(r_input_data)
print(r_t_data)

[[ 38.  64. 270.]
 [155. 111. 173.]
 [260. 133. 248.]
 ...
 [ 27.  49.  98.]
 [111. 174. 107.]
 [208. 290. 197.]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


## 가중치 W, 편향 B  생성 ##
1. 정규분포를 따르는 랜덤수로 생성   (np.random.randn()을 사용하면 "평균 = 0, 표준편차 = 1"인 랜덤수가 생성됨.)
    
    0.01을 곱해주는 이유: w의 초기값이크면 gradient vanishing 문제가 발생할 수 있음.

    따라서 초기 가중치 값에 0.01을 곱해 표준편차를 0.01로 만들어줌.


In [306]:
weight_init_std=1
params={}
params['W1']=weight_init_std*np.random.randn(3,4)
params['B1']=np.zeros(4)
params['W2']=weight_init_std*np.random.randn(4,4)
params['B2']=np.zeros(4)
params['W3']=weight_init_std*np.random.randn(4,3)
params['B3']=np.zeros(3)


2. constant로 생성

In [282]:
# params['W1']=np.zeros((3,4))
# params['B1']=np.zeros(4)
# params['W2']=np.zeros((4,4))
# params['B2']=np.zeros(4)
# params['W3']=np.zeros((4,3))
# params['B3']=np.zeros(3)

# params['W1'].fill(0.1)
# params['W2'].fill(0.1)
# params['W3'].fill(0.1)


3. 균일분포를 따르는 랜덤수로 생성

In [283]:
# weight_init_std=0.01
# params={}
# params['W1']=np.random.uniform(-1,1,(3,4))
# params['B1']=np.zeros(4)
# params['W2']=np.random.uniform(-1,1,(4,4))
# params['B2']=np.zeros(4)
# params['W3']=np.random.uniform(-1,1,(4,3))
# params['B3']=np.zeros(3)
# print(params)

## 가중치 W, 편향 B weight_bias.npz 파일에 저장

np.savez() --> 여러개의 배열을 1개의 압축되지 않은 *.npz 포맷 파일로 저장하기 (Save several arrays into a single file in uncompressed .npz format)

In [289]:
np.savez("../simple_network/Data/weight_bias.npz", **params)

##### dictionary형 변수 csv파일에 저장

In [307]:
# open a file in write mode
with open('../simple_network/Data/weight_bias.csv', 'w', newline='') as f:
    # create a csv writer
    writer = csv.writer(f)
    # write the keys of the dictionary as the header row
    writer.writerow(params.keys())
    # write the values of the dictionary as a single row
    writer.writerow(params.values())

# new_params={}
# new_params['W1']=params['W1']
# new_params['W2']=params['W2']
# new_params['W3']=params['W3']
# with open('eeeeeeeee.csv', 'w', newline='') as csvfile:
#     # Create a CSV writer
#     writer = csv.writer(csvfile)

#     # Write the values of the W1, W2, and W3 arrays to the CSV file
#     for key, value in new_params.items():
#         writer.writerows(value)

npz 변수 파일 불러오기

np.load("파일명",allow_pickle=True)

In [286]:
my_dictionary = np.load("../simple_network/Data/weight_bias.npz", allow_pickle=True)
paaa=dict(my_dictionary)
print(paaa)
print(params)

{'W1': array([[-0.01001585, -0.01367141,  0.00324272,  0.01296843],
       [ 0.01303907, -0.00629541,  0.01125086,  0.01020688],
       [-0.00769062,  0.00965486,  0.01910086,  0.02126137]]), 'B1': array([0., 0., 0., 0.]), 'W2': array([[ 0.00883512, -0.00668806,  0.00908893,  0.01812063],
       [-0.0025648 ,  0.00157932, -0.01517578,  0.00087346],
       [ 0.00944193, -0.01055826,  0.00127665, -0.0296165 ],
       [ 0.021103  ,  0.0058067 , -0.00734992, -0.00285864]]), 'B2': array([0., 0., 0., 0.]), 'W3': array([[-9.26541726e-03, -5.07083010e-03, -1.87769771e-02],
       [ 5.79218276e-03,  1.46035095e-02,  1.49302147e-02],
       [-7.57663035e-03,  1.07737976e-02, -1.18589639e-02],
       [-5.33776501e-03,  2.76365415e-03, -7.24315098e-05]]), 'B3': array([0., 0., 0.])}
{'W1': array([[-0.01001585, -0.01367141,  0.00324272,  0.01296843],
       [ 0.01303907, -0.00629541,  0.01125086,  0.01020688],
       [-0.00769062,  0.00965486,  0.01910086,  0.02126137]]), 'B1': array([0., 0., 0., 0.